In [1]:
from Parameters import EnviromentParameters, TrainingParameters
from Data import XESDatasetWithResource
import tensorflow as tf

In [2]:
dataset = XESDatasetWithResource(file_path= EnviromentParameters.BPI2020DatasetWithResource.file_path, preprocessed_folder_path=EnviromentParameters.BPI2020DatasetWithResource.preprocessed_foldr_path, preprocessed_df_type= EnviromentParameters.BPI2020DatasetWithResource.preprocessed_df_type,include_types=TrainingParameters.bpi2012.BPI2012_include_types)


| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012_with_resource/AOW 


In [3]:
caseids, padded_data_traces, lengths, padded_data_resources, batch_amount, padded_target_traces = dataset.collate_fn([0,1,2])

In [33]:
activity_embedding_dim = 32
resource_embedding_dim = 32
lstm_hidden = 16
dropout = .2
dense_dim = 32

In [5]:
activity_embeddding = tf.keras.layers.Embedding(
    input_dim = len(dataset.vocab),
    output_dim= activity_embedding_dim,
    mask_zero= True,
)

resource_embedding = tf.keras.layers.Embedding(
    input_dim = len(dataset.resources),
    output_dim = resource_embedding_dim,
    mask_zero = True
)



In [6]:
activity_emb_out = activity_embeddding(padded_data_traces, training=True)

In [7]:
resource_emb_out = resource_embedding(padded_data_resources, training=True) 

In [8]:
mask = activity_embeddding.compute_mask(padded_data_traces)

In [27]:
activity_lstm = tf.keras.layers.LSTM(
            lstm_hidden,
            return_sequences=True,
            return_state=True,
        )

activity_lstm_sec =  tf.keras.layers.LSTM(
            lstm_hidden,
            return_sequences=True,
            return_state=True,
        )

resource_lstm = tf.keras.layers.LSTM(
            lstm_hidden,
            return_sequences=True,
            return_state=True,
        )

resource_lstm_sec = tf.keras.layers.LSTM(
            lstm_hidden,
            return_sequences=True,
            return_state=True,
        )

In [37]:
padded_data_traces.shape

(3, 38)

In [28]:
activity_lstm_out, a_h_out, a_c_out = activity_lstm(activity_emb_out, training=False, mask=mask, initial_state=None )

In [29]:
activity_lstm_out_sec, a_h_out_sec, a_c_out_sec = activity_lstm_sec(activity_lstm_out, training=False, mask=mask, initial_state=None )

In [30]:
resources_lstm_out, r_h_out, r_c_out = resource_lstm(resource_emb_out, training=False, mask=mask, initial_state=None )

In [31]:
resources_lstm_out_sec, r_h_out_sec, r_c_out_sec = resource_lstm_sec(resources_lstm_out, training=False, mask=mask, initial_state=None )

In [14]:
amount_to_concate = tf.repeat(tf.expand_dims(tf.expand_dims(tf.constant(batch_amount),axis=1),axis=2), resources_lstm_out.shape[1], axis=1)

In [32]:
concat_out = tf.concat([activity_lstm_out_sec, resources_lstm_out_sec, amount_to_concate], axis = -1)

In [34]:
out_net = tf.keras.models.Sequential(
    [
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(dense_dim),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(len(dataset.vocab)),
    ]
)

In [35]:
out = out_net(concat_out)

In [38]:
t = "A"

In [39]:
t if t else "C"

'A'

In [59]:
amount_to_concate.shape

TensorShape([3, 38, 1])

In [60]:
activity_lstm_out.shape

TensorShape([3, 38, 16])

38